In [25]:
from datasets import load_dataset
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
import re

In [4]:
references = load_dataset('csv', data_files=r'/Users/adrianfolge/Documents/lokal:skole/Master/data/synthetic_data/question_with_answers.csv', split="train[:50]")

In [5]:
references["Question"]

['Hva er datoen for vedtaket av Kommunedelplan for sentrum av bystyret?',
 'Hva er hovedintensjonene i planen som er beskrevet i dokumentet?',
 'Hva er hovedintensjonene bak planarbeidet for videreutvikling av Kristiansund sentrum?',
 'Hva er kravet til antall sykkelparkeringsplasser for en skole med 10 årsverk ifølge tabellen?',
 'Hva er kravene for å sette opp skilt og reklameinnretninger i henhold til plan- og bygningsloven?',
 'Hva er minimumsarealet som kreves for private uteoppholdsarealer for boenheter med inngang fra felles trappeoppgang?',
 'Hva er minimumsarealet som skal avsettes til lekeplass for områder som ikke ligger i hensynssone kulturminne og hvor det etableres 5-20 nye boenheter?',
 'Hva er formålet med områdene som er definert som "Fritids- og turistformål" og "Næringsbebyggelse" i dokumentet?',
 'Hva er retningslinjene for bevaring av førkrigsbebyggelsen i hensynssonene H570_100-199?',
 'Hva er kravene til taktekking og vinduer for alle trehus i området i henhold t

In [27]:
loader = DirectoryLoader('data/', glob="**/*.pdf", show_progress=True, loader_cls=UnstructuredFileLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
databases = {}
for doc in documents:
    source = doc.metadata['source']
    match = re.search(r'\/([A-Za-z_]+)\.pdf', source)
    if match:
        municipality_name = match.group(1)
    docs = text_splitter.split_documents([doc])
    embeddings = SentenceTransformerEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    db = FAISS.from_documents(docs, embeddings)
    databases[municipality_name] = db

100%|██████████| 10/10 [00:06<00:00,  1.43it/s]
Created a chunk of size 1147, which is longer than the specified 500
Created a chunk of size 1570, which is longer than the specified 500
Created a chunk of size 639, which is longer than the specified 500
Created a chunk of size 610, which is longer than the specified 500
Created a chunk of size 1008, which is longer than the specified 500
Created a chunk of size 545, which is longer than the specified 500
Created a chunk of size 536, which is longer than the specified 500
Created a chunk of size 583, which is longer than the specified 500
Created a chunk of size 1162, which is longer than the specified 500
Created a chunk of size 607, which is longer than the specified 500
Created a chunk of size 1206, which is longer than the specified 500
Created a chunk of size 697, which is longer than the specified 500
Created a chunk of size 734, which is longer than the specified 500
Created a chunk of size 916, which is longer than the specified

In [31]:
docs

[Document(page_content='PLANBESTEMMELSER §1 GENERELT 1.0 Formål Reguleringsplanen skal legge til rette for boligbebyggelse med tilhørende uteoppholdsareal. 1.1 Planavgrensning Det regulerte området er vist med plangrense på plankart sist revidert 18.1.19 i målestokk 1:1000 Planen vil erstatte deler av reguleringsplan ’Støver øst og Støver vest, vedtatt 10.02.2000. 1.2 Planområdets arealformål Området reguleres til følgende formål, jfr plan- og bygningsloven § 12-5: Bebyggelse og anlegg: - Konsentrert småhusbebyggelse (BK1-BK2) - Annet uteoppholdsareal (f_AU1) - Renovasjonsanlegg(f_RE1) Samferdselsanlegg og teknisk infrastruktur - Veg(f_SKV1) - Gatetun (f_GT1) -Annen veggrunn (f_SVT1-2) - Parkering (f_P1-2) Grønnstruktur - Friområde (f_F1)', metadata={'source': 'data/pdfs/Bodø.pdf'}),
 Document(page_content='Side 1 av 5\n\nformannskapssekretær\n\nPLANBESTEMMELSER FOR DETALJREGULERINGSPLAN FOR INNERDAMMEN, STØVER, GNR 54/4, 55/107 Siste behandling i PNM komiteen dato: Vedtatt av Bodø bys